<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from pymystem3 import Mystem
import spacy
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

from sklearn.metrics import make_scorer
import numpy as np



In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Проверим на дубликаты 

In [5]:
data.duplicated().sum()

0

Функция для очистки текста.

In [6]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]',' ',text).split())

Функция для лематизации

In [7]:
parse = spacy.load('en_core_web_sm')

def lemmatize(s, exceptions={"PUNCT", "SPACE"}): 
    doc = parse(s)
    return " ".join([token.lemma_ for token in doc if token.pos_ not in exceptions])

Тут умерает ядро , пару раз сработало пока писал код , как только начал тестить всё умерло. 
Что делать О_О?

In [8]:
data_lemm = data['text'].apply(lambda x: clear_text(x))
data_lemm = data_lemm.apply(lambda x: lemmatize(x))
data_lemm.head()

0    explanation why the edit make under my usernam...
1    d aww he match this background colour I m seem...
2    hey man I m really not try to edit war it s ju...
3    More I can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: text, dtype: object

## Обучение

Разделим данные на тестовые и тренировачные. 

In [9]:
train , test  = train_test_split(data_lemm, test_size=0.25, random_state=12345)

Векторезируем данные.

In [10]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

tf_idf_train = count_tf_idf.fit_transform(train) 
tf_idf_test = count_tf_idf.transform(test) 

Выделим признаки и цели.

In [11]:
features_train = tf_idf_train
features_test = tf_idf_test
target_train = data.loc[train.index]['toxic']
target_test = data.loc[test.index]['toxic']

Функция для подбора парметров.

In [12]:
results = pd.DataFrame(columns=['model','parametrs','f1'])     
score = make_scorer(f1_score, greater_is_better=False)

def trainer (model_train, parameters):
    
    
    model = GridSearchCV(model_train, parameters, cv=3, scoring='f1', n_jobs=-1)
    results = pd.DataFrame(columns=['model','parametrs','f1'])
    
    model.fit(features_train,target_train)

    results = results.append({'model': model_train,
                              'parametrs': model.best_params_,
                              'f1': -model.best_score_}, ignore_index=True)

    return results

LogisticRegression

In [13]:
parametrs = {}
model = LogisticRegression(max_iter = 200, class_weight='balanced')
results = results.append(trainer(model,parametrs),  ignore_index=True)

DecisionTreeClassifier

In [14]:
parametrs = {'max_depth': range(2, 5),'random_state': [12345]}
model = DecisionTreeClassifier(class_weight='balanced')
results = results.append(trainer(model,parametrs),  ignore_index=True)

RandomForestClassifier

In [16]:
parametrs = {'n_estimators':[100,150,200],'max_depth': range(2, 5),'random_state': [12345]}
model = RandomForestClassifier(class_weight='balanced')
results = results.append(trainer(model,parametrs),  ignore_index=True)

Лучшая модель.

In [18]:
pd.options.display.max_colwidth = 90
results[results['f1'].min() == results['f1']]

,model,parametrs,f1
0,"LogisticRegression(class_weight='balanced', max_iter=200)",{},-0.748348


Проверим на тестовой выборке.

In [19]:
model = LogisticRegression(class_weight='balanced',random_state=12345, max_iter = 200) 
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(f1_score(target_test,predictions))

0.7537450156266838


## Выводы

Лучшая модель LogisticRegression f1 = 0.7537